In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

from abc import ABC, abstractmethod
from collections import defaultdict

# Decision Tree Algorithm

1. Инициализация
    - Имеется набор данных $D$
    - Инициализируется дерево $T$
2. Рост дерева
    - Рекурсивно для каждой ноды до достижения критерия останова:
        - Для каждого признака $f$ и порогового значения $s$ набор данных в ноде $D_t$ разбивается на две подвыборки:
            $$D_L=\{x \in D | x_f \leqslant s\}; \ D_R=\{x \in D | x_f > s\}$$
        - Для каждого разбиения считается мера чистоты $I$.
        - Для разбиения выбираются такие $f$ и $s$, которые минимизируют $I$: $argmin_{f,s}\frac{|D_L|}{|D_t|}I(D_L) + \frac{|D_R|}{|D_t|}I(D_R) $
        - Производится разбиение $D_t$ на $D_L$ и $D_R$
4. Предсказание
    - Для каждого наблюдения осуществляется спуск от корня дерева до листьев в соотвествии с обученными $f$ и $s$ для узловых нод.
    Исходя из этого алгоритм дерева решений не может экстраполировать
    - Предсказание опредляется большинством голосов для классификации и средним для регрессии 

# Implementation

In [68]:
class Node():
    '''
    Класс, сохраняющий основную информацию о ноде дерева
    '''
    
    def __init__(self) -> None:
        self.feature = None        # индекс признака для сплита в ноде
        self.split = None          # значения для сплита
        self.left_node = None      # левая нода
        self.right_node = None     # правая нода
        self.leaf_value = None     # значение листа, если нода листовая
        self.impurity = None       # чистота ноды
        self.p = None              # пропорция наблюдений в ноде по отношению к N
        
    def set_params(self, feature: int, split: float, impurity: float = None, p: float = None) -> None:
        '''
        Устанавливает основные параметры ноды
        '''
        self.feature = feature
        self.split = split
        self.impurity = impurity
        self.p = p
        
    def get_params(self) -> tuple[int, float]:
        '''
        Возвращает индекс признака и пороговое значание для сплита
        '''
        return self.feature, self.split    
        
    def set_child_nodes(self, left_node: 'instance', right_node: 'instance') -> None:
        '''
        Устанавливает дочерние ноды
        '''
        self.left_node  = left_node
        self.right_node = right_node
        
    def get_left_node(self) -> 'instance':
        '''
        Возвращает левую ноду
        '''
        return self.left_node
    
    def get_right_node(self) -> 'instance':
        '''
        Возвращает правую ноду
        '''
        return self.right_node

In [69]:
class BaseDecisionTree(ABC):
    '''
    Базовый класс для дерева решений. Реализован в упрощенной форме для образовательных целей.
    
    Для классификации мера чистоты - entropy
    Для регрессии мера чистоты - mse
    '''
    
    def __init__(self, max_depth: int = 2, min_samples_split: int = 2) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    @abstractmethod
    def _calculate_impurity(self, data: np.array) -> float:
        '''
        Рассчитывает чистоту выборки/ноды.
        '''
        pass
        
    def _calculate_split_impurity(self, data_left: np.array, data_right: np.array) -> float:
        '''
        Расчитывает чистоту дочерних нод после сплита
        '''
        n = data_left.shape[0] + data_right.shape[0]
        p_left = data_left.shape[0] / n
        p_right = data_right.shape[0] / n
        return p_left * self._calculate_impurity(data_left) + p_right * self._calculate_impurity(data_right)
    
    @abstractmethod
    def _calculate_leaf_value(self, data: np.array) -> float:
        '''
        Расчитывает значение листа
        '''
        pass
    
    @staticmethod
    def get_splits(feature_values: np.array) -> np.array:
        '''
        Возвращает возможные значения для разбиения
        '''
        unique_values = np.unique(feature_values)
        return (unique_values[:-1] + unique_values[1:]) / 2
    
    def _get_best_split(self, data: np.array) -> tuple[int, float, float]:
        '''
        Возвращает наиболее оптимальный признак и значение для сплита на основании чистоты в дочерних нодах
        '''
        min_impurity = None
        best_feature = None
        best_split = None
        for feature in range(data[:, :-1].shape[1]):
            splits = self.get_splits(data[:, feature])
            for split in splits:
                data_left = data[data[:, feature] <= split]
                data_right = data[data[:, feature] > split]
                if (data_left.shape[0] == 0) or (data_right.shape[0] == 0):
                    continue
                split_impurity = self._calculate_split_impurity(data_left, data_right)
                if (min_impurity is None) or (split_impurity < min_impurity):
                    min_impurity = split_impurity
                    best_feature = feature
                    best_split = split
        return best_feature, best_split, self._calculate_impurity(data)

    @staticmethod
    def make_split(data: np.array, feature: int, split: float) -> tuple[np.array, np.array]:
        '''
        Разделяет выборку на две части по признаку и значению
        '''
        return data[data[:, feature] <= split], data[data[:, feature] > split]
    
    def _grow_tree(self, node: 'class', data: np.array, depth: int) -> None:
        '''
        Строит дерево от корневой ноды
        '''
        if (depth == self.max_depth) or (data.shape[0] < self.min_samples_split) or (np.unique(data[:, -1]).shape[0] == 1):
            node.leaf_value = self._calculate_leaf_value(data)
            node.impurity = self._calculate_impurity(data)
            node.p = data.shape[0] / self.n
        else:
            data_left = None
            data_right = None
            best_feature, best_split, node_impurity = self._get_best_split(data)
            data_left, data_right = self.make_split(data, best_feature, best_split)
            node.set_params(best_feature, best_split, node_impurity, data.shape[0] / self.n)
            left_node = Node()
            right_node = Node()
            node.set_child_nodes(left_node, right_node)
            self._grow_tree(node.get_left_node(), data_left, depth + 1)
            self._grow_tree(node.get_right_node(), data_right, depth + 1)
            
    def _traverse_tree(self, node: 'class', data: np.array) -> float:
        '''
        Возвращает значение листа для наблюдения. Используется в предсказании.
        '''
        if node.leaf_value is None:
            best_feture, best_split = node.get_params()
            if data[best_feture] <= best_split:
                return self._traverse_tree(node.get_left_node(), data)
            else:
                return self._traverse_tree(node.get_right_node(), data)
        else:
            return node.leaf_value   

    def fit(self, X: np.array, y: np.array) -> None:
        data = np.c_[X, y]
        self.n, self.p = X.shape
        self.tree = Node()
        self._grow_tree(self.tree, data, 0)
        self.feature_importances = np.zeros(self.p)
        self._compute_feature_importance(self.tree)
        self.feature_importances = self.feature_importances / np.sum(self.feature_importances)

    def predict(self, X: np.array) -> np.array:
        predictions = []
        for row in range(X.shape[0]):
            prediction = self._traverse_tree(self.tree, X[row, :])
            predictions.append(prediction)
        return np.array(predictions)
    
    def _compute_feature_importance(self, node: 'class') -> None:
        '''
        Расчитывает важность для каждого признака по всему дереву
        '''
        if node.leaf_value is None:
            self.feature_importances[node.feature] += self._get_importance(node)
            self._compute_feature_importance(node.get_left_node())
            self._compute_feature_importance(node.get_right_node())
            
    def _get_importance(self, node: 'class') -> float:
        '''
        Расчитывает важность признака в конкретной ноде
        '''
        left_node = node.get_left_node()
        right_node = node.get_right_node()
        return node.p * node.impurity - left_node.impurity * left_node.p - right_node.impurity * right_node.p

In [70]:
class DecisionTreeClassifier_(BaseDecisionTree):
    
    def _calculate_impurity(self, data: np.array) -> float:
        class_counts = np.unique(data[:, -1], return_counts=True)[1]
        probabilities = class_counts / sum(class_counts)
        return -sum(probabilities * np.log2(probabilities+1e-9))
    
    def _calculate_leaf_value(self, data: np.array) -> int:
        classes = np.unique(data[:, -1], return_counts=True)
        plural_class_idx = np.argmax(classes[1])
        return classes[0][plural_class_idx]

In [71]:
class DecisionTreeRegressor_(BaseDecisionTree):

    def _calculate_impurity(self, data: np.array) -> float:
        y = data[:, -1]
        y_mean = np.mean(y)
        return np.sum((y - y_mean)**2) / y.shape[0]
    
    def _calculate_leaf_value(self, data: np.array) -> int:
        y = data[:, -1]
        return np.mean(y)

In [72]:
X, y = make_classification(n_samples=1000, n_features=3, n_informative=3,  n_redundant=0, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
custom_model = DecisionTreeClassifier_(max_depth=4, min_samples_split=2)
sklearn_model = DecisionTreeClassifier(max_depth=4, min_samples_split=2, criterion='entropy')
custom_model.fit(X_train, y_train)
sklearn_model.fit(X_train, y_train)
custom_pred = custom_model.predict(X_test)
sklearn_pred = sklearn_model.predict(X_test)

print('\n---Results---\n')
print(f'custom accuracy: {accuracy_score(y_test, custom_pred).round(2)}')
print(f'sklearn accuracy: {accuracy_score(y_test, sklearn_pred).round(2)}')


---Results---

custom accuracy: 0.9
sklearn accuracy: 0.9


In [74]:
custom_model.feature_importances

array([0.79534016, 0.1469735 , 0.05768634])

In [75]:
sklearn_model.feature_importances_

array([0.79534016, 0.1469735 , 0.05768634])

In [76]:
X, y = make_regression(n_samples=1000, n_features=3, n_informative=2, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
model_custom = DecisionTreeRegressor_(max_depth=4, min_samples_split=2)
model_sklearn = DecisionTreeRegressor(max_depth=4, min_samples_split=2, criterion='squared_error')
model_custom.fit(X_train, y_train)
model_sklearn.fit(X_train, y_train)

custom_pred = model_custom.predict(X_test)
sklearn_pred = model_sklearn.predict(X_test)

print(f'custom MSE: {mean_squared_error(y_test, custom_pred).round(3)}')
print(f'sklearn MSE: {mean_squared_error(y_test, sklearn_pred).round(3)}')

custom MSE: 2325.927
sklearn MSE: 2325.927


In [78]:
model_custom.feature_importances

array([0.60794398, 0.39205602, 0.        ])

In [79]:
model_sklearn.feature_importances_

array([0.60794398, 0.39205602, 0.        ])

# Random Forest Algorithm

1. Инициализация
    - Имеется набор данных $D$
    - Устанавливается размер ансамбля $B$, т.е. кол-во решающих деревьев в лесу
    - Генерируется $B$ бутстрапированных выборок 
2. Рост леса
    - Для каждой бустрапированной выборки $b=1,...,B$:
        - Инициализируется дерево $T_b$, обучающаеся на выборке $b$. 
        - Для каждого дерева $T_b$:
            - Выбирается случайная подвыборка признаков $F_{sub}$ из всего набора признаков $F$. Обычно $F_{sub}=\sqrt{F}$
            - Осуществляется рост дерева в соотвествии с общим алгоритмом дерева решений на основании $F_{sub}$
            - Обученое дерево решений $T_b$ сохраняется
            - (Опционально) Сохраняются out-of-bag наблюдения для дерева $T_b$, т.е. наблюдения, которые не попали в выборку $b$.
    - (Опционально) Out-of-bag пердсказания усредняются и расчитывается out-of-bag error
    - Формируется случайный лес, состоящий из $T_1,...,B$ решающих деревьев
3. Предсказание
    - Каждое наблюдение пропускается через ансамбль деревьев $T_1,...,B$, формируется $B$ предсказаний
    - Финальное предсказание опредляется большинством голосов для классификации и средним для регрессии

# Implementation

In [83]:
class BaseRandomForest(ABC):
    '''
    Базовый класс для алгоритма случайного леса. Реализован в упрощенной форме для образовательных целей.
    '''
    
    def __init__(
        self, 
        n_estimators: int = 100,
        max_depth: int = None, 
        min_samples_split: int = 2,
        max_features = 'sqrt'
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self._decision_trees = []
        self.oob_predictions = defaultdict(list)
        self.oob_score = None


    @abstractmethod
    def _init_decision_tree(self) -> 'class':
        '''
        Инициализирует инстанс дерева решений
        '''
        pass
    
    @abstractmethod
    def _compute_oob_score(self, y_train: np.array, oob_predictions: dict[list]) -> float:
        '''
        Рассчитывает оценку на oob выборке
        '''
        pass
    
    def fit(self, X: np.array, y: np.array) -> None:
        '''
        Обучает ансамбль деревьев и рассчитывает oob оценку
        '''
        n_samples, p_features = X.shape
        row_idx = list(range(n_samples))
        column_idx = list(range(p_features))
        for b in range(self.n_estimators):
            sample_row_idx = np.random.choice(row_idx, size=n_samples, replace=True)
            X_sampled, y_sampled = X[sample_row_idx, :], y[sample_row_idx]
            decision_tree = self._init_decision_tree()
            decision_tree.fit(X_sampled, y_sampled)
            self._decision_trees.append(decision_tree)
            oob_row_idx = list(set(row_idx) - set(sample_row_idx))
            if oob_row_idx:
                oob_prediction = decision_tree.predict(X[oob_row_idx])
                for idx, prediction in zip(oob_row_idx, oob_prediction):
                    self.oob_predictions[idx].append(prediction)
        self.oob_score = self._compute_oob_score(y, self.oob_predictions)
        
    def predict(self, X: np.array) -> np.array:
        tree_predictions = []
        for decision_tree in self._decision_trees:
            prediction = decision_tree.predict(X).reshape(-1,1)
            tree_predictions.append(prediction)
        tree_predictions = np.concatenate(tree_predictions, axis=1)
        return self._compute_ensemble_prediction(tree_predictions)
    
    @abstractmethod
    def _compute_ensemble_prediction(self, tree_predictions: np.array) -> np.array:
        '''
        Рассчитывает финальное предсказание на основани мн-ва предсказаний ансамбля решающих деревьев
        '''
        pass

In [84]:
class RandomForestClassifier_(BaseRandomForest):
        
    def __init__(
        self, 
        n_estimators: int = 100,
        max_depth: int = None, 
        min_samples_split: int = 2,
        max_features = 'sqrt',
        criterion = 'entropy'
    ):
        super().__init__(n_estimators, max_depth, min_samples_split, max_features)
        self.criterion = criterion
        
        
    def _init_decision_tree(self):
        return DecisionTreeClassifier(
                    max_depth = self.max_depth, 
                    min_samples_split = self.min_samples_split, 
                    max_features = self.max_features,
                    criterion = self.criterion
        )
    
    def _compute_oob_score(self, y_train, oob_predictions):
        true_labels = np.array([y_train[idx] for idx in oob_predictions.keys()])
        oob_prediction = np.array([np.bincount(predictions).argmax() for predictions in oob_predictions.values()])
        return accuracy_score(true_labels, oob_prediction)
    
    def _compute_ensemble_prediction(self, tree_predictions):
        return np.apply_along_axis(lambda predictions: np.bincount(predictions).argmax(), axis=1, arr=tree_predictions.astype(int))

In [85]:
class RandomForestRegressor_(BaseRandomForest):
        
    def __init__(
        self, 
        n_estimators: int = 100,
        max_depth: int = None, 
        min_samples_split: int = 2,
        max_features = 'sqrt',
        criterion = 'squared_error'
    ):
        super().__init__(n_estimators, max_depth, min_samples_split, max_features)
        self.criterion = criterion
        
        
    def _init_decision_tree(self):
        return DecisionTreeRegressor(
                    max_depth = self.max_depth, 
                    min_samples_split = self.min_samples_split, 
                    max_features = self.max_features,
                    criterion = self.criterion
        )
    
    def _compute_oob_score(self, y_train, oob_predictions):
        true_labels = np.array([y_train[idx] for idx in oob_predictions.keys()])
        oob_prediction = np.array([np.mean(predictions) for predictions in oob_predictions.values()])
        return r2_score(true_labels, oob_prediction)
    
    def _compute_ensemble_prediction(self, tree_predictions):
        return np.mean(tree_predictions, axis=1)

In [86]:
X, y = make_classification(n_samples=10000, n_features=10, n_informative=6,  n_redundant=0, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
custom_model = RandomForestClassifier_(n_estimators=100, max_depth=None, min_samples_split=2)
sklearn_model = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, oob_score=True)
custom_model.fit(X_train, y_train)
sklearn_model.fit(X_train, y_train)
custom_pred = custom_model.predict(X_test)
sklearn_pred = sklearn_model.predict(X_test)

print('\n---Results---\n')
print(f'custom accuracy: {accuracy_score(y_test, custom_pred).round(2)}')
print(f'sklearn accuracy: {accuracy_score(y_test, sklearn_pred).round(2)}')
print(f'custom oob score: {custom_model.oob_score.round(3)}')
print(f'sklearn oob score: {sklearn_model.oob_score_.round(3)}')


---Results---

custom accuracy: 0.95
sklearn accuracy: 0.95
custom oob score: 0.954
sklearn oob score: 0.951


In [88]:
X, y = make_regression(n_samples=10000, n_features=10, n_informative=6, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
custom_model = RandomForestRegressor_(n_estimators=100, max_depth=None, min_samples_split=2)
sklearn_model = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2, max_features='sqrt', oob_score=True)
custom_model.fit(X_train, y_train)
sklearn_model.fit(X_train, y_train)
custom_pred = custom_model.predict(X_test)
sklearn_pred = sklearn_model.predict(X_test)

print('\n---Results---\n')
print(f'custom MSE: {mean_squared_error(y_test, custom_pred).round(2)}')
print(f'sklearn MSE: {mean_squared_error(y_test, sklearn_pred).round(2)}')
print(f'custom oob score: {custom_model.oob_score.round(3)}')
print(f'sklearn oob score: {sklearn_model.oob_score_.round(3)}')


---Results---

custom MSE: 1136.98
sklearn MSE: 1109.18
custom oob score: 0.931
sklearn oob score: 0.933
